# 2150188401(2) Artificial intelligence Final Project: Finetuning_BERT

Copyright (C) Computer Science & Engineering, Soongsil University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. Written by Haneul Pyeon, November 2024.

BERT(Bidirectional Encoder Representations from Transformers) is a groundbreaking model in the NLP domain. This tutorial provides a step-by-step guide on how to fine-tune the lightweight BERT variant using Hugging Face's transformers library for text classification tasks.<br>

This is about BERT (Devlin et al., 2018).<br>
https://arxiv.org/abs/1810.04805

The code below are based on the following link. <br>
https://medium.com/@khang.pham.exxact/text-classification-with-bert-7afaacc5e49b


### Fine-tune the model
1. Design your model's prediction head
2. Finetune the model by changing the hyperparameters.
3. You will get a score based on the your (hidden) test accuracy for text classification (ranking-based).  

### Submitting your work:
<font color=red>**DO NOT clear the final outputs**</font> so that we can grade both your code and results.  


Now proceed to the code.


## Install libraries

In [13]:
import os

In [14]:
!python3 -m pip install pandas
!python3 -m pip install transformers

### import libraries

In [15]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

### Specify your GPU number if necessary

In [16]:
%env CUDA_VISIBLE_DEVICES = 0

if torch.cuda.is_available() is True:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

env: CUDA_VISIBLE_DEVICES=0


## Preparing dataset

link : https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

1. Download the dataset from attached link.
2. Move the downloaded zip file under the "data" directory and then unzip the zip file.
3. Run the following cell

In [17]:
def load_imdb_data(data_file_path):
    if os.path.exists(data_file_path):
        df = pd.read_csv(data_file_path)
        texts = df['review'].tolist()
        labels = [1 if sentiment == "positive" else 0 for sentiment in df['sentiment'].tolist()]
        return texts, labels
    else:
        raise FileNotFoundError(f"The file '{data_file_path}' does not exist.")

data_file_path = './data/IMDB Dataset Train.csv'
texts, labels = load_imdb_data(data_file_path)

## Dataset class

In [18]:
class CustomTextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_seq_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

## Classifier head for BERT( Design your model's prediction head )

In [34]:
class CustomBERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(CustomBERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        ######################## TO-DO ########################

        # 1. Hidden Layer
        self.hidden_layer = nn.Linear(self.bert.config.hidden_size, 768)
        self.relu = nn.ReLU()

        # 2. Dropout Layer
        self.dropout = nn.Dropout(p = 0.20)

        # 3. Output Layer
        self.output_layer = nn.Linear(768, num_classes)

        ######################## TO-DO ########################

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        ######################## TO-DO ########################

        # 1. Hidden Layer + Activation
        x = self.hidden_layer(pooled_output)
        x = self.relu(x)

        # 2. Dropout
        x = self.dropout(x)

        # 3. Output Layer
        logits = self.output_layer(x)

        ######################## TO-DO ########################
        return logits

## train and evaluation method

In [35]:
def train_model(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in tqdm(data_loader, desc="Train"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        ######################## TO-DO ########################
        loss = nn.CrossEntropyLoss()(outputs, labels)
        ######################## TO-DO ########################
        loss.backward()
        optimizer.step()
        scheduler.step()

def evaluate_model(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

## Hyper-parameter settings

In [36]:
# Set up parameters
# Hint: generally, 5 ~ 10 epochs will be enough.
bert_model_name = 'bert-base-uncased'
num_classes = 2
######################## TO-DO ########################
max_seq_length = 500
batch_size = 8
num_epochs = 10
learning_rate = 1e-5
######################## TO-DO ########################

## get data utils

In [37]:
######################## DO NOT CHANGE ########################
train_texts, val_texts, train_labels, val_labels = \
train_test_split(texts, labels, test_size=0.2, random_state=42)
######################## DO NOT CHANGE ########################

tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = CustomTextClassificationDataset(train_texts, train_labels, tokenizer, max_seq_length)
val_dataset = CustomTextClassificationDataset(val_texts, val_labels, tokenizer, max_seq_length)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

## Define model, optimizer and scheduler

In [38]:
model = CustomBERTClassifier(bert_model_name, num_classes).to(device)
######################## TO-DO ########################
optimizer = AdamW(model.parameters(), lr = learning_rate, weight_decay = 0.02)
######################## TO-DO ########################
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Train model and save best model

In [40]:
veval_acc = 0
for epoch in range(num_epochs):
    model_path = './finetuned_bert.pth'
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_model(model, train_dataloader, optimizer, scheduler, device)
    accuracy, report = evaluate_model(model, val_dataloader, device)
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(report)

    if eval_acc < accuracy:
        torch.save(model.state_dict(), model_path)
        print('Saved Trained Model.')
        eval_acc = accuracy

Epoch 1/5


Validation: 100%|██████████| 1000/1000 [04:26<00:00,  3.75it/s]


Validation Accuracy: 0.9387
              precision    recall  f1-score   support

           0       0.95      0.93      0.94      3911
           1       0.93      0.95      0.94      4089

    accuracy                           0.94      8000
   macro avg       0.94      0.94      0.94      8000
weighted avg       0.94      0.94      0.94      8000

Saved Trained Model.
Epoch 2/5


Validation: 100%|██████████| 1000/1000 [04:27<00:00,  3.74it/s]


Validation Accuracy: 0.9405
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      3911
           1       0.94      0.95      0.94      4089

    accuracy                           0.94      8000
   macro avg       0.94      0.94      0.94      8000
weighted avg       0.94      0.94      0.94      8000

Saved Trained Model.
Epoch 3/5


...

Validation Accuracy: 0.9417
              precision    recall  f1-score   support

           0       0.95      0.93      0.94      3911
           1       0.94      0.95      0.94      4089

    accuracy                           0.94      8000
   macro avg       0.94      0.94      0.94      8000
weighted avg       0.94      0.94      0.94      8000

Saved Trained Model.
Epoch 4/5


Validation: 100%|██████████| 1000/1000 [04:27<00:00,  3.73it/s]


Validation Accuracy: 0.9419
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      3911
           1       0.94      0.95      0.94      4089

    accuracy                           0.94      8000
   macro avg       0.94      0.94      0.94      8000
weighted avg       0.94      0.94      0.94      8000

Saved Trained Model.
Epoch 5/5


Validation: 100%|██████████| 1000/1000 [04:27<00:00,  3.75it/s]

Validation Accuracy: 0.9419
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      3911
           1       0.94      0.95      0.94      4089

    accuracy                           0.94      8000
   macro avg       0.94      0.94      0.94      8000
weighted avg       0.94      0.94      0.94      8000

